In [1]:
# import necessary libraries
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def calc_L(c_val, freq_val):
    '''
    Calculate an inductance from a capacitance and a resonant frequency.
    '''
    l_denom = 4 * (np.pi**2) * (freq_val**2) * c_val
    l_val = 1 / l_denom
    return l_val

In [3]:
def calc_C(l_val, freq_val):
    '''
    Calculate a capacitance from an inductance and a resonant frequency.
    '''
    c_denom = 4 * (np.pi**2) * (freq_val**2) * l_val
    c_val = 1 / c_denom
    return c_val

In [4]:
def calc_f(c_val, l_val):
    '''
    Calculate a resonant frequency from a capacitance and an inductance
    '''
    f_val = (1 / (2*np.pi)) * (1 / np.sqrt(c_val * l_val))
    return f_val

In [5]:
def calc_frange_midpoint(l_val, c_upper, c_lower):
    '''
    Calculate the midpoint of a range of frequencies determined by a range of capacitance values.
    '''
    f_lower = calc_f(c_upper, l_val)
    f_upper = calc_f(c_lower, l_val)
    midpoint = f_lower + 0.5*(f_upper - f_lower)
    return midpoint

In [6]:
def optimize(f_val, c_upper, c_lower, samples):
    '''
    Determine an inductance value that will produce the desired resonant frequency as close as possible
    to the midpoint of a range of frequencies determined by a range of capacitance values.
    '''
    # calculate range of inductances that will contain resonant frequency
    l_min = calc_L(c_upper, f_val)
    l_max = calc_L(c_lower, f_val)

    # create an array of possible inductance values
    # increasing `samples` increases the resolution
    l_values = np.linspace(l_min, l_max, samples)

    n = len(l_values)
  
    mid_values = np.zeros(n) # store midpoints of frequency ranges
    dist_values = np.zeros(n) # store differences between midpoints and resonant frequency
  
    # for each possible inductance value, 
    # find the midpoint of the associated frequency range a
    # and the difference between midpoint and resonant frequency
    for i in range(n):
        current_l = l_values[i]
        mid_values[i] = calc_frange_midpoint(current_l, c_upper, c_lower)
        dist_values[i] = np.abs(mid_values[i] - f_val)

    # find the index of the inductance whose midpoint frequency is closest to the resonant frequency
    l_index = np.argmin(dist_values)
    
    # inductance to produce resonant frequency centered in frequency range
    optimal_l = l_values[l_index]
    return optimal_l

In [7]:
# constants
f = 1.4e6
c_max = 660e-12
c_min = 60e-12
H_to_mH = 1e6

In [8]:
optimal_l = optimize(f, c_max, c_min, 100000)
optimal_l_mH = optimal_l * H_to_mH
print('Optimal L =', np.round(optimal_l_mH,2), 'microH')

Optimal L = 91.22 microH


In [9]:
f_min = calc_f(c_max, optimal_l)
f_max = calc_f(c_min, optimal_l)
print('minimum frequency =', f_min, 'Hz')
print('desired frequency =', f, 'Hz')
print('maximum frequency =', f_max, 'Hz')

minimum frequency = 648653.25724257 Hz
desired frequency = 1400000.0 Hz
maximum frequency = 2151339.473315486 Hz


## equations that the functions below are based on HERE

In [10]:
def calc_turns(l_val, a_val, tpi):
    '''
    Calculate number of coils from an inductance, radius, and turns-per-inch associated with wire used.
    '''
    # solve for roots of equation - print as a check
    poly = [a_val**2, l_val*(-10/tpi), -9*a_val*l_val]
    turns = np.roots(poly)
    print('possible number of turns:', turns, '\n')
    
    # extract valid roots
    if len(turns) == 0:
        # if there are no roots, print error and return nothing
        print("Uh oh! Problem calculating turns - no roots!")
        return
    else:
        # check for positive roots
        pos_index = np.where(turns >= 0)
        if len(pos_index) == 0:
            # if there are no positive roots, print error and return nothing
            print("Uh oh! Problem calculating turns - no positive roots!")
            return
        elif len(pos_index) > 1:
            # if there are multiple positive roots, return them all
            print("Heads up! Multiple roots!")
            return turns
        else:
            # if there is only one positive root, return it
            return turns[pos_index]
        
    return turns

In [11]:
def calc_l_fromturns(n_val, a_val, tpi):
    '''
    Calculate inductance from number of coils, radius, and turns-per-inch of wire used
    '''
    l_num = (n_val**2)*(a_val**2)
    l_denom = (10*(1/tpi)*n_val) + (9*a_val)
    l_val = l_num / l_denom
    return l_val

In [12]:
# constants
A = 1
turns_per_inch = 30
pF_to_F = 1e-12

In [13]:
# calculate number of turns needed to produce desired inductance
N = calc_turns(optimal_l_mH, A, turns_per_inch)
N_int = int(N[0]+0.5) # account for truncation in int conversion
print('Inductor should be wound with', N_int, 'turns')

# calculate the actual inductance from the determined number of coils
actual_l_mH = calc_l_fromturns(N_int, A, turns_per_inch)
actual_l = actual_l_mH / H_to_mH
print('to produce an inductance of', actual_l_mH, 'microH')

# check that it is approximately equal to optimal inductance
percent_diff = 100*np.abs(optimal_l - actual_l) / actual_l
print('This differs from optimal inductance by', np.round(percent_diff,2), '%')

possible number of turns: [ 47.63822715 -17.23288698] 

Inductor should be wound with 48 turns
to produce an inductance of 92.16 microH
This differs from optimal inductance by 1.02 %
